In [ ]:
# -*- coding: utf-8 -*-
# ======================================================
# Datatur YTD -> Pie Plotly con estilo corporativo
# ADAPTADO A PLOTLY/STREAMLIT con estilos dinámicos
# ======================================================

import io, re, zipfile, unicodedata
from datetime import datetime
from typing import Optional, Tuple, List

import numpy as np
import pandas as pd
import requests
import plotly.graph_objects as go
import streamlit as st 

# ----------------- Constantes -----------------
DATATUR_ZIP_URL = "https://datatur.sectur.gob.mx/Documentos%20compartidos/CUADRO_DGAC.zip"
UA = {"User-Agent":"Mozilla/5.0 (compatible; DataPipeline/1.4; +https://github.com)"}

# Tipografía preferida
FONT_STACK = "Aptos Light, Aptos, Arial, sans-serif"

# Tamaño y márgenes para exportación
FIG_WIDTH = 800
FIG_HEIGHT = 500
LEFT_MARGIN = 220

# Expresiones p/ detectar cuadro y columnas
TITLE_RE = re.compile(
    r"pasajeros\s+transportados\s+en\s+vuelos\s+internacionales\s+por\s+principales\s+aerolineas",
    re.IGNORECASE
)
AEROL_RE = re.compile(r"aerol[ií]neas?", re.IGNORECASE)

YTD_TAG_RE = re.compile(
    r"(ene(?:ro)?|feb|mar|abr|may|jun|jul|ago|sept?|oct|nov|dic)"
    r".{0,6}"
    r"(?:a|–|-|al)?"
    r".{0,6}"
    r"(ene(?:ro)?|feb|mar|abr|may|jun|jul|ago|sept?|oct|nov|dic)",
    re.IGNORECASE
)
YEAR_RE = re.compile(r"\(?\b(20\d{2})\b\)?")
PART_RE = re.compile(r"participaci[oó]n?\s*%?", re.IGNORECASE)

REGION_KEYS = {
    "total mexicanas": "Mexicanas",
    "total estadounidenses": "Estadounidenses",
    "total canadienses": "Canadienses",
    "total europeas": "Europeas",
    "total centro y sudamericanas": "Centro/Sudamericanas",
    "total asiaticas": "Asiáticas",
    "total": "TOTAL",
}

# ---------------------------------------------
# Adaptación de Estilo para Streamlit
# ---------------------------------------------
DEFAULT_COLOR_TOPS = ["#889064", "#0e1c2c", "#ff9f18"]
DEFAULT_REST_PALETTE = ["#b5b9ad", "#2a3b55", "#ffb955", "#d0d4c8", "#3d4f6b", "#ffd08a", "#7a8467", "#1a2a40"]

PALETTE = globals().get('active_palette', DEFAULT_COLOR_TOPS + DEFAULT_REST_PALETTE)
FONT = globals().get('active_font', FONT_STACK)

FONT_FAMILY_PLOTLY = FONT
COLOR_TOPS = PALETTE[:3]
REST_PALETTE = PALETTE[3:]


# ----------------- Utilerías -----------------
def zstrip(s: str) -> str:
    s = unicodedata.normalize("NFD", str(s))
    s = "".join(c for c in s if unicodedata.category(c) != "Mn")
    s = re.sub(r"\s+", " ", s.strip().lower())
    s = s.replace("t o t a l", "total")
    return s

def download_zip(url: str) -> bytes:
    r = requests.get(url, headers=UA, timeout=90)
    r.raise_for_status()
    return r.content

def ffill_list(lst: List[str]) -> List[str]:
    out, cur = [], ""
    for x in lst:
        s = (str(x) if x is not None else "").strip()
        if s.lower() != "nan" and s != "":
            cur = s
        out.append(cur)
    return out

def find_title_blocks(df: pd.DataFrame) -> List[Tuple[int,int]]:
    blocks = []
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            val = df.iat[i,j]
            if pd.isna(val): continue
            if TITLE_RE.search(zstrip(val)):
                blocks.append((i,j))
    return blocks

def reconstruct_with_two_header_rows(raw: pd.DataFrame, hdr_row: int, sec_row: int) -> Tuple[pd.DataFrame, List[str]]:
    max_cols = raw.shape[1]
    top = [raw.iat[hdr_row, j] if j < raw.shape[1] else "" for j in range(max_cols)]
    bot = [raw.iat[sec_row, j] if j < raw.shape[1] else "" for j in range(max_cols)]
    top = [re.sub(r"\s+", " ", str(x).strip()) for x in top]
    bot = [re.sub(r"\s+", " ", str(x).strip()) for x in bot]
    top_ff = ffill_list(top)
    cols = []
    for j, (t, b) in enumerate(zip(top_ff, bot)):
        if b and (YEAR_RE.search(b) or PART_RE.search(b) or "Variaci" in b):
            cols.append((f"{t} {b}").strip() if t else b)
        else:
            cols.append(t if t else (b if b else f"col_{j}"))
    start = sec_row + 1
    end = start
    while end < raw.shape[0]:
        c0 = zstrip(raw.iat[end,0]) if end < raw.shape[0] else ""
        if "fuente:" in c0 or c0.startswith("notas:"):
            break
        end += 1
    body = raw.iloc[start:end, :len(cols)].copy()
    body.columns = cols[:body.shape[1]]
    return body, cols

def read_sheet_candidate(xls: pd.ExcelFile, sheet: str) -> Optional[pd.DataFrame]:
    raw = pd.read_excel(xls, sheet_name=sheet, header=None, engine="openpyxl")
    blocks = find_title_blocks(raw)
    if blocks:
        for (r, _c) in blocks:
            for offset in range(1, 6):
                hdr_row = r + offset
                if hdr_row >= raw.shape[0]: continue
                row_vals = raw.iloc[hdr_row,:].astype(str).tolist()
                if any(AEROL_RE.search(v) for v in row_vals):
                    sec_row = hdr_row + 1 if hdr_row + 1 < raw.shape[0] else hdr_row
                    body, cols = reconstruct_with_two_header_rows(raw, hdr_row, sec_row)
                    aerol_cols = [k for k in body.columns if AEROL_RE.search(k)]
                    if not aerol_cols:
                        best_col, best_hits = None, 0
                        for k in body.columns:
                            vals = body[k].astype(str).map(zstrip)
                            hits = sum(v in REGION_KEYS for v in vals)
                            if hits > best_hits:
                                best_col, best_hits = k, hits
                        if best_col and best_hits >= 5:
                            body.rename(columns={best_col:"Aerolíneas"}, inplace=True)
                        else:
                            continue
                    else:
                        body.rename(columns={aerol_cols[0]:"Aerolíneas"}, inplace=True)
                    vals = body["Aerolíneas"].astype(str).map(zstrip)
                    body = body.loc[vals.isin(REGION_KEYS.keys())].copy()
                    if not body.empty and body.shape[0] >= 6:
                        return body
    best_hits = 0; best = None
    for j in range(raw.shape[1]):
        vals = raw.iloc[:, j].astype(str).map(zstrip)
        hits = vals.isin(REGION_KEYS.keys()).sum()
        if hits > best_hits:
            best_hits = hits; best = vals
    if best_hits >= 5:
        first_lab_row = int(best[best.isin(REGION_KEYS.keys())].index.min())
        hdr_row = max(0, first_lab_row - 2)
        sec_row = min(raw.shape[0]-1, hdr_row + 1)
        body, cols = reconstruct_with_two_header_rows(raw, hdr_row, sec_row)
        best_col, best_hits = None, 0
        for k in body.columns:
            vals = body[k].astype(str).map(zstrip)
            hits = vals.isin(REGION_KEYS.keys()).sum()
            if hits > best_hits:
                best_col, best_hits = k, hits
        if best_col and best_hits >= 5:
            body.rename(columns={best_col:"Aerolíneas"}, inplace=True)
            vals = body["Aerolíneas"].astype(str).map(zstrip)
            body = body.loc[vals.isin(REGION_KEYS.keys())].copy()
            if not body.empty and body.shape[0] >= 6:
                return body
    return None

def extract_year(text: str) -> Optional[int]:
    if not isinstance(text, str): text = str(text)
    m = YEAR_RE.search(text)
    return int(m.group(1)) if m else None

def pick_latest_ytd_columns(columns: List[str], *, verbose=False) -> Tuple[Optional[str], Optional[str], Optional[str], Optional[int]]:
    cols = [str(c) for c in columns]
    candidates = []
    for c in cols:
        if YTD_TAG_RE.search(c) and YEAR_RE.search(c):
            yr = extract_year(c)
            if yr: candidates.append((c, yr))
    if candidates:
        candidates.sort(key=lambda x: x[1])
        latest_year = candidates[-1][1]
        col_latest = next(c for c,y in candidates if y == latest_year)
        prevs = [c for c,y in candidates if y < latest_year]
        col_prev = prevs[-1] if prevs else None
        col_part = None
        for c in cols:
            if str(latest_year) in c and PART_RE.search(c):
                col_part = c; break
        return col_prev, col_latest, col_part, latest_year

    year_cols = []
    for c in cols:
        yr = extract_year(c)
        if yr: year_cols.append((c, yr))
    if year_cols:
        year_cols.sort(key=lambda x: x[1])
        latest_year = year_cols[-1][1]
        col_latest = next(c for c,y in year_cols if y == latest_year)
        prevs = [c for c,y in year_cols if y < latest_year]
        col_prev = prevs[-1][0] if prevs else None
        col_part = None
        for c in cols:
            if str(latest_year) in c and PART_RE.search(c):
                col_part = c; break
        return col_prev, col_latest, col_part, latest_year
    return None, None, None, None

def to_number(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
        .str.replace(r"[^\d\-,\.]", "", regex=True)
        .str.replace(".", "", regex=False)
        .str.replace(",", "", regex=False)
        .replace({"": None, "-": None})
        .astype(float)
    )

def get_series(df: pd.DataFrame, colname: str) -> pd.Series:
    obj = df[colname]
    if isinstance(obj, pd.DataFrame): return obj.iloc[:, -1]
    return obj

def _select_participation_series(df: pd.DataFrame, latest_year: int):
    part_cols = [c for c in df.columns if PART_RE.search(str(c))]
    if not part_cols: return None
    year_cols = [c for c in part_cols if str(latest_year) in str(c)]
    chosen = year_cols[-1] if year_cols else part_cols[-1]
    ser = df[chosen]
    if isinstance(ser, pd.DataFrame): ser = ser.iloc[:, -1]
    return ser

# ----------------- Pipeline principal -----------------
@st.cache_data(ttl=3600)
def run_datatur_analysis():
    try:
        content = download_zip(DATATUR_ZIP_URL)
    except Exception as e:
        raise RuntimeError(f"Fallo al descargar el ZIP de Datatur: {e}")

    zf = zipfile.ZipFile(io.BytesIO(content))
    excel_files = [m for m in zf.infolist() if m.filename.lower().endswith((".xlsx", ".xls"))]
    if not excel_files:
        raise RuntimeError("El ZIP no contiene archivos Excel.")

    parsed = None
    for memb in excel_files:
        try:
            data = io.BytesIO(zf.read(memb))
            xls = pd.ExcelFile(data, engine="openpyxl")
            for sh in xls.sheet_names:
                body = read_sheet_candidate(xls, sh)
                if body is not None:
                    parsed = body; break
            if parsed is not None: break
        except Exception: pass

    if parsed is None:
        raise RuntimeError("No se encontró la tabla de 'Internacionales' en los Excel del ZIP.")

    parsed["Region"] = parsed["Aerolíneas"].astype(str).map(zstrip).map(REGION_KEYS.get)
    col_prev, col_latest, col_part, latest_year = pick_latest_ytd_columns(parsed.columns.tolist())
    if col_latest is None:
        raise RuntimeError("No se detectó un bloque de columnas con año/YTD en los encabezados.")

    parsed[col_latest] = to_number(get_series(parsed, col_latest))
    if col_prev and col_prev in parsed.columns:
        parsed[col_prev] = to_number(get_series(parsed, col_prev))

    part_series = _select_participation_series(parsed, latest_year) if col_part else None
    if part_series is not None:
        ser_str = part_series.astype(str).str.replace("%", "", regex=False).str.replace(",", ".", regex=False)
        ser_num = pd.to_numeric(ser_str, errors="coerce")
        max_val = ser_num.dropna().max()
        part = ser_num * 100.0 if (pd.notna(max_val) and max_val <= 1.0) else ser_num
    else:
        tmp = parsed[parsed["Region"] != "TOTAL"]
        latest_vals_tmp = to_number(get_series(tmp, col_latest))
        s = latest_vals_tmp.sum()
        latest_vals_all = to_number(get_series(parsed, col_latest))
        part = (latest_vals_all / s * 100.0) if (s and np.isfinite(s) and s > 0) else pd.Series(np.nan, index=parsed.index)

    prev_year = extract_year(col_prev) if col_prev else None

    out = parsed.copy()
    if prev_year:
        out[f"{prev_year}_YTD"] = out[col_prev]
    out[f"{latest_year}_YTD"] = out[col_latest]
    out["Participacion_%"] = part
    if prev_year:
        denom = out[col_prev]
        with np.errstate(divide='ignore', invalid='ignore'):
            out["Variacion_rel_%"] = (out[col_latest] - denom) / denom * 100.0

    order = ["Estadounidenses","Mexicanas","Canadienses","Europeas","Centro/Sudamericanas","Asiáticas","TOTAL"]
    cols = ["Region"]
    if prev_year: cols.append(f"{prev_year}_YTD")
    cols += [f"{latest_year}_YTD", "Participacion_%"]
    if prev_year: cols.append("Variacion_rel_%")
    out = out.set_index("Region").reindex(order).reset_index()[cols]

    # ----------------- Pie con Plotly -----------------
    pie_df = out[out["Region"] != "TOTAL"].copy()
    vals = pie_df[f"{latest_year}_YTD"].fillna(0).values
    labels = pie_df["Region"].values

    ranks_desc = np.argsort(-vals)
    rank_map = {idx: rank for rank, idx in enumerate(ranks_desc)}

    colors = [
        (COLOR_TOPS[rank_map[i]] if rank_map[i] < 3 else REST_PALETTE[(rank_map[i]-3) % len(REST_PALETTE)])
        for i in range(len(vals))
    ]
    pulls = [0.06 if rank_map[i] == 0 else 0.0 for i in range(len(vals))]

    fig = go.Figure(
        go.Pie(
            labels=labels,
            values=vals,
            textinfo="text",
            texttemplate="%{percent:.1%}",
            textposition="outside",
            textfont=dict(size=12, family=FONT_FAMILY_PLOTLY),
            marker=dict(colors=colors, line=dict(color="white", width=1)),
            pull=pulls,
            sort=False,
            direction="clockwise",
            hovertemplate="<b>%{label}</b><br>%{value:,.0f} pasajeros<br>%{percent:.1%}<extra></extra>"
        )
    )

    m_period = re.search(r"\(([^)]*?)\)", col_latest)
    periodo = (m_period.group(1).strip() if m_period else (YTD_TAG_RE.search(col_latest).group(0).upper() if YTD_TAG_RE.search(col_latest) else "YTD"))

    fig.update_layout(
        # 1. Título Centrado
        title=dict(
            text=f"Participación {latest_year} pasajeros transportados internacionales<br>",
            x=0.5,
            xanchor='center',
            font=dict(size=18, family=FONT_FAMILY_PLOTLY)
        ),
        font=dict(family=FONT_FAMILY_PLOTLY, size=12),
        legend=dict(
            title=f"Participación {latest_year}",
            orientation="v",
            y=0.5, yanchor="middle",
            x=1.02, xanchor="left",
            font=dict(size=12, family=FONT_FAMILY_PLOTLY),
            tracegroupgap=6,
            itemwidth=40
        ),
        # 2. Aumento de margen inferior (b) para la fuente
        margin=dict(l=LEFT_MARGIN, r=140, t=100, b=100),
        autosize=False,
        width=FIG_WIDTH,
        height=FIG_HEIGHT,
        showlegend=True
    )

    for tr in fig.data:
        if isinstance(tr, go.Pie):
            tr.domain = dict(x=[0.0, 0.88], y=[0.05, 0.95])

    # 3. Anotación de Fuente (Inferior Izquierda)
    fig.add_annotation(
        text="Fuente: Secretaría de Turismo (Datatur)",
        xref="paper", yref="paper",
        x=0,      
        y=-0.1,   
        showarrow=False,
        xanchor='left',
        yanchor='top',
        font=dict(size=11, color="gray", family=FONT_FAMILY_PLOTLY)
    )

    return fig, out, f"{periodo} {latest_year}"

# ----------------- Ejecución Streamlit -----------------

st.title("Internacionales por Aerolínea (Datatur)")

if st.button("Recargar datos (Datatur)"):
    st.cache_data.clear()

try:
    fig, out_df, periodo_str = run_datatur_analysis()

    st.subheader(f"Distribución de Pasajeros por Región — {periodo_str}")
    st.plotly_chart(fig, use_container_width=True)

    # 4. Tabla de datos debajo de la gráfica
    st.markdown("### Tabla de Datos Detallada")
    st.dataframe(out_df, use_container_width=True)

except RuntimeError as e:
    st.error(f"Error fatal al procesar los datos de Datatur: {e}")
except Exception as e:
    st.error(f"Ocurrió un error inesperado durante la ejecución: {e}")